In [140]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from sqlalchemy import create_engine
from sklearn.preprocessing import MinMaxScaler
import datetime
import getpass

%matplotlib inline

In [4]:
df = pd.read_csv('df.csv')
temp = pd.read_csv('temp.csv')
top95 = temp[(temp['p_click_sum'] > temp['p_click_sum'].quantile(0.95)) & (temp['shop_click'] > temp['shop_click'].quantile(0.95))]
top95

,Unnamed: 0,uid,pay_sum,pay_point,pay_free,p_cnt,avg_price,category_cnt,follower_cnt,avg_upload_term,shop_click,avg_prod_click,p_click_sum,zzim,comment_cnt,normal_review_cnt,cert_review_cnt,review_cnt,point_group
60,226,124402,66000.0,66000.0,0.0,116.0,51000.0000,20.0,1159,16.556522,6963.0,46.0,53749.0,480.0,190.0,60.0,52.0,112.0,4
86,322,187078,66000.0,66000.0,0.0,91.0,383107.6813,5.0,648,22.611111,5777.0,76.0,52369.0,329.0,56.0,25.0,44.0,69.0,4
98,367,216864,66000.0,66000.0,0.0,100.0,72003.0000,22.0,303,19.676768,5834.0,65.0,35901.0,647.0,111.0,19.0,13.0,32.0,4
149,534,333950,734349.0,734349.0,0.0,61.0,168516.3934,9.0,1541,28.033333,9053.0,226.0,59434.0,332.0,230.0,54.0,130.0,184.0,5
173,617,400112,99000.0,99000.0,0.0,146.0,42034.2055,46.0,1071,15.062069,5098.0,73.0,48830.0,729.0,189.0,41.0,42.0,83.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2808,10559,9494505,577392.0,577392.0,0.0,122.0,110844.2623,24.0,379,17.254237,8176.0,101.0,55483.0,549.0,82.0,4.0,14.0,18.0,5
2831,10640,9578622,440060.0,440060.0,0.0,221.0,48110.8597,31.0,176,8.781818,6627.0,161.0,57831.0,714.0,48.0,18.0,22.0,40.0,5
2868,10803,9739724,641680.0,641680.0,0.0,186.0,63247.3118,33.0,213,8.744565,7024.0,126.0,46134.0,899.0,45.0,4.0,12.0,16.0,5
2886,10903,9836425,706482.0,706482.0,0.0,34.0,212029.4118,5.0,336,40.363636,5863.0,113.0,53108.0,343.0,25.0,2.0,14.0,16.0,5


In [8]:
uids = top95['uid'].tolist()

In [91]:
q = f'''
SELECT uid,
       first_date AS ad_first_date
FROM   (SELECT uid,
               Min(date) AS first_date
        FROM   ((SELECT uid,
                        Min(created_at) AS date
                 FROM   ad_power_up
                 WHERE  uid IN ( 124402, 187078, 216864, 333950,
                                 400112, 436861, 651784, 743783,
                                 861179, 1156106, 1279768, 1411576,
                                 1711078, 2260704, 2368168, 2511601,
                                 2619878, 2683035, 3007742, 3017955,
                                 3232973, 3338363, 3345895, 3402156,
                                 3462965, 3617987, 3809243, 3812367,
                                 3866299, 3869908, 4002904, 4075620,
                                 4255482, 4261654, 4324142, 4324226,
                                 4498563, 4520924, 4532474, 4538177,
                                 4674335, 4678892, 4713848, 4748732,
                                 4754326, 5029391, 5140808, 5181775,
                                 5247560, 5307910, 5330079, 5383378,
                                 5409449, 5409692, 5691009, 5886399,
                                 5902202, 5964192, 6102698, 6252355,
                                 6327523, 6455668, 6648834, 6728006,
                                 6781262, 6787025, 6788274, 6805575,
                                 6868557, 6886070, 7004811, 7006458,
                                 7195361, 7247270, 7292822, 7303057,
                                 7563119, 7668781, 7675857, 7865374,
                                 7866004, 7945915, 7989118, 8182612,
                                 8276420, 8395704, 8845498, 8851436,
                                 8959243, 9012884, 9016331, 9046433,
                                 9114504, 9212044, 9366942, 9481304,
                                 9494505, 9578622, 9739724, 9836425, 9979854 )
                 GROUP  BY uid)
                UNION ALL
                (SELECT uid,
                        Min(create_at) AS date
                 FROM   ad_super_up
                 WHERE  uid IN ( 124402, 187078, 216864, 333950,
                                 400112, 436861, 651784, 743783,
                                 861179, 1156106, 1279768, 1411576,
                                 1711078, 2260704, 2368168, 2511601,
                                 2619878, 2683035, 3007742, 3017955,
                                 3232973, 3338363, 3345895, 3402156,
                                 3462965, 3617987, 3809243, 3812367,
                                 3866299, 3869908, 4002904, 4075620,
                                 4255482, 4261654, 4324142, 4324226,
                                 4498563, 4520924, 4532474, 4538177,
                                 4674335, 4678892, 4713848, 4748732,
                                 4754326, 5029391, 5140808, 5181775,
                                 5247560, 5307910, 5330079, 5383378,
                                 5409449, 5409692, 5691009, 5886399,
                                 5902202, 5964192, 6102698, 6252355,
                                 6327523, 6455668, 6648834, 6728006,
                                 6781262, 6787025, 6788274, 6805575,
                                 6868557, 6886070, 7004811, 7006458,
                                 7195361, 7247270, 7292822, 7303057,
                                 7563119, 7668781, 7675857, 7865374,
                                 7866004, 7945915, 7989118, 8182612,
                                 8276420, 8395704, 8845498, 8851436,
                                 8959243, 9012884, 9016331, 9046433,
                                 9114504, 9212044, 9366942, 9481304,
                                 9494505, 9578622, 9739724, 9836425, 9979854 )
                 GROUP  BY uid)
                UNION ALL
                (SELECT uid,
                        Min(create_at) AS date
                 FROM   ad_super_up_shop
                 WHERE  uid IN ( 124402, 187078, 216864, 333950,
                                 400112, 436861, 651784, 743783,
                                 861179, 1156106, 1279768, 1411576,
                                 1711078, 2260704, 2368168, 2511601,
                                 2619878, 2683035, 3007742, 3017955,
                                 3232973, 3338363, 3345895, 3402156,
                                 3462965, 3617987, 3809243, 3812367,
                                 3866299, 3869908, 4002904, 4075620,
                                 4255482, 4261654, 4324142, 4324226,
                                 4498563, 4520924, 4532474, 4538177,
                                 4674335, 4678892, 4713848, 4748732,
                                 4754326, 5029391, 5140808, 5181775,
                                 5247560, 5307910, 5330079, 5383378,
                                 5409449, 5409692, 5691009, 5886399,
                                 5902202, 5964192, 6102698, 6252355,
                                 6327523, 6455668, 6648834, 6728006,
                                 6781262, 6787025, 6788274, 6805575,
                                 6868557, 6886070, 7004811, 7006458,
                                 7195361, 7247270, 7292822, 7303057,
                                 7563119, 7668781, 7675857, 7865374,
                                 7866004, 7945915, 7989118, 8182612,
                                 8276420, 8395704, 8845498, 8851436,
                                 8959243, 9012884, 9016331, 9046433,
                                 9114504, 9212044, 9366942, 9481304,
                                 9494505, 9578622, 9739724, 9836425, 9979854 )
                 GROUP  BY uid)
                UNION ALL
                (SELECT uid,
                        Min(created_at) AS date
                 FROM   ad_up_plus
                 WHERE  uid IN ( 124402, 187078, 216864, 333950,
                                 400112, 436861, 651784, 743783,
                                 861179, 1156106, 1279768, 1411576,
                                 1711078, 2260704, 2368168, 2511601,
                                 2619878, 2683035, 3007742, 3017955,
                                 3232973, 3338363, 3345895, 3402156,
                                 3462965, 3617987, 3809243, 3812367,
                                 3866299, 3869908, 4002904, 4075620,
                                 4255482, 4261654, 4324142, 4324226,
                                 4498563, 4520924, 4532474, 4538177,
                                 4674335, 4678892, 4713848, 4748732,
                                 4754326, 5029391, 5140808, 5181775,
                                 5247560, 5307910, 5330079, 5383378,
                                 5409449, 5409692, 5691009, 5886399,
                                 5902202, 5964192, 6102698, 6252355,
                                 6327523, 6455668, 6648834, 6728006,
                                 6781262, 6787025, 6788274, 6805575,
                                 6868557, 6886070, 7004811, 7006458,
                                 7195361, 7247270, 7292822, 7303057,
                                 7563119, 7668781, 7675857, 7865374,
                                 7866004, 7945915, 7989118, 8182612,
                                 8276420, 8395704, 8845498, 8851436,
                                 8959243, 9012884, 9016331, 9046433,
                                 9114504, 9212044, 9366942, 9481304,
                                 9494505, 9578622, 9739724, 9836425, 9979854 )
                 GROUP  BY uid)
                UNION ALL
                (SELECT uid,
                        Min(created_at) AS date
                 FROM   ad_today_deal
                 WHERE  uid IN ( 124402, 187078, 216864, 333950,
                                 400112, 436861, 651784, 743783,
                                 861179, 1156106, 1279768, 1411576,
                                 1711078, 2260704, 2368168, 2511601,
                                 2619878, 2683035, 3007742, 3017955,
                                 3232973, 3338363, 3345895, 3402156,
                                 3462965, 3617987, 3809243, 3812367,
                                 3866299, 3869908, 4002904, 4075620,
                                 4255482, 4261654, 4324142, 4324226,
                                 4498563, 4520924, 4532474, 4538177,
                                 4674335, 4678892, 4713848, 4748732,
                                 4754326, 5029391, 5140808, 5181775,
                                 5247560, 5307910, 5330079, 5383378,
                                 5409449, 5409692, 5691009, 5886399,
                                 5902202, 5964192, 6102698, 6252355,
                                 6327523, 6455668, 6648834, 6728006,
                                 6781262, 6787025, 6788274, 6805575,
                                 6868557, 6886070, 7004811, 7006458,
                                 7195361, 7247270, 7292822, 7303057,
                                 7563119, 7668781, 7675857, 7865374,
                                 7866004, 7945915, 7989118, 8182612,
                                 8276420, 8395704, 8845498, 8851436,
                                 8959243, 9012884, 9016331, 9046433,
                                 9114504, 9212044, 9366942, 9481304,
                                 9494505, 9578622, 9739724, 9836425, 9979854 )
                 GROUP  BY uid)
                UNION ALL
                (SELECT uid,
                        Min(created_at) AS date
                 FROM   ad_today_deal_plus
                 WHERE  uid IN ( 124402, 187078, 216864, 333950,
                                 400112, 436861, 651784, 743783,
                                 861179, 1156106, 1279768, 1411576,
                                 1711078, 2260704, 2368168, 2511601,
                                 2619878, 2683035, 3007742, 3017955,
                                 3232973, 3338363, 3345895, 3402156,
                                 3462965, 3617987, 3809243, 3812367,
                                 3866299, 3869908, 4002904, 4075620,
                                 4255482, 4261654, 4324142, 4324226,
                                 4498563, 4520924, 4532474, 4538177,
                                 4674335, 4678892, 4713848, 4748732,
                                 4754326, 5029391, 5140808, 5181775,
                                 5247560, 5307910, 5330079, 5383378,
                                 5409449, 5409692, 5691009, 5886399,
                                 5902202, 5964192, 6102698, 6252355,
                                 6327523, 6455668, 6648834, 6728006,
                                 6781262, 6787025, 6788274, 6805575,
                                 6868557, 6886070, 7004811, 7006458,
                                 7195361, 7247270, 7292822, 7303057,
                                 7563119, 7668781, 7675857, 7865374,
                                 7866004, 7945915, 7989118, 8182612,
                                 8276420, 8395704, 8845498, 8851436,
                                 8959243, 9012884, 9016331, 9046433,
                                 9114504, 9212044, 9366942, 9481304,
                                 9494505, 9578622, 9739724, 9836425, 9979854 )
                 GROUP  BY uid)
                UNION ALL
                (SELECT uid,
                        Min(created_at) AS date
                 FROM   ad_shop_popular
                 WHERE  uid IN ( 124402, 187078, 216864, 333950,
                                 400112, 436861, 651784, 743783,
                                 861179, 1156106, 1279768, 1411576,
                                 1711078, 2260704, 2368168, 2511601,
                                 2619878, 2683035, 3007742, 3017955,
                                 3232973, 3338363, 3345895, 3402156,
                                 3462965, 3617987, 3809243, 3812367,
                                 3866299, 3869908, 4002904, 4075620,
                                 4255482, 4261654, 4324142, 4324226,
                                 4498563, 4520924, 4532474, 4538177,
                                 4674335, 4678892, 4713848, 4748732,
                                 4754326, 5029391, 5140808, 5181775,
                                 5247560, 5307910, 5330079, 5383378,
                                 5409449, 5409692, 5691009, 5886399,
                                 5902202, 5964192, 6102698, 6252355,
                                 6327523, 6455668, 6648834, 6728006,
                                 6781262, 6787025, 6788274, 6805575,
                                 6868557, 6886070, 7004811, 7006458,
                                 7195361, 7247270, 7292822, 7303057,
                                 7563119, 7668781, 7675857, 7865374,
                                 7866004, 7945915, 7989118, 8182612,
                                 8276420, 8395704, 8845498, 8851436,
                                 8959243, 9012884, 9016331, 9046433,
                                 9114504, 9212044, 9366942, 9481304,
                                 9494505, 9578622, 9739724, 9836425, 9979854 )
                 GROUP  BY uid)) AS ads
        GROUP  BY uid) AS firsts '''

dates = pd.read_sql(q, con=mysql)

In [93]:
# subtract 90days from the day users used an ad for the first time
dates['begin'] = dates['ad_first_date'] - datetime.timedelta(days=90)

In [95]:
q = f'''
SELECT id,
       target_id  AS pid,
       seller_uid AS uid,
       updated
FROM   item_click_log6
WHERE  updated BETWEEN '2016-02-04 00:00:00' AND '2019-12-11 10:30:09'
       AND seller_uid IN ( '124402', '187078', '216864', '333950',
                           '400112', '436861', '651784', '743783',
                           '861179', '1156106', '1279768', '1411576',
                           '1711078', '2260704', '2368168', '2511601',
                           '2619878', '2683035', '3007742', '3017955',
                           '3232973', '3338363', '3345895', '3402156',
                           '3462965', '3617987', '3809243', '3812367',
                           '3866299', '3869908', '4002904', '4075620',
                           '4255482', '4261654', '4324142', '4324226',
                           '4498563', '4520924', '4532474', '4538177',
                           '4674335', '4678892', '4713848', '4748732',
                           '4754326', '5029391', '5140808', '5181775',
                           '5247560', '5307910', '5330079', '5383378',
                           '5409449', '5409692', '5691009', '5886399',
                           '5902202', '5964192', '6102698', '6252355',
                           '6327523', '6455668', '6648834', '6728006',
                           '6781262', '6787025', '6788274', '6805575',
                           '6868557', '6886070', '7004811', '7006458',
                           '7195361', '7247270', '7292822', '7303057',
                           '7563119', '7668781', '7675857', '7865374',
                           '7866004', '7945915', '7989118', '8182612',
                           '8276420', '8395704', '8845498', '8851436',
                           '8959243', '9012884', '9016331', '9046433',
                           '9114504', '9212044', '9366942', '9481304',
                           '9494505', '9578622', '9739724', '9836425', '9979854'
                         ) 
'''
item_click = pd.read_sql(q, con=redshift)

In [24]:
q = f'''
SELECT id,
       target_id AS uid,
       updated
FROM   shop_click_log
WHERE  updated BETWEEN '2016-02-04 00:00:00' AND '2019-12-11 10:30:09'
       AND target_id IN ( '124402', '187078', '216864', '333950',
                          '400112', '436861', '651784', '743783',
                          '861179', '1156106', '1279768', '1411576',
                          '1711078', '2260704', '2368168', '2511601',
                          '2619878', '2683035', '3007742', '3017955',
                          '3232973', '3338363', '3345895', '3402156',
                          '3462965', '3617987', '3809243', '3812367',
                          '3866299', '3869908', '4002904', '4075620',
                          '4255482', '4261654', '4324142', '4324226',
                          '4498563', '4520924', '4532474', '4538177',
                          '4674335', '4678892', '4713848', '4748732',
                          '4754326', '5029391', '5140808', '5181775',
                          '5247560', '5307910', '5330079', '5383378',
                          '5409449', '5409692', '5691009', '5886399',
                          '5902202', '5964192', '6102698', '6252355',
                          '6327523', '6455668', '6648834', '6728006',
                          '6781262', '6787025', '6788274', '6805575',
                          '6868557', '6886070', '7004811', '7006458',
                          '7195361', '7247270', '7292822', '7303057',
                          '7563119', '7668781', '7675857', '7865374',
                          '7866004', '7945915', '7989118', '8182612',
                          '8276420', '8395704', '8845498', '8851436',
                          '8959243', '9012884', '9016331', '9046433',
                          '9114504', '9212044', '9366942', '9481304',
                          '9494505', '9578622', '9739724', '9836425', '9979854'
                        ) 
'''
shop_click = pd.read_sql(q, con=redshift)

In [100]:

dates['uid'] = dates['uid'].astype(int)
item_click['uid'] = item_click['uid'].astype(int)
shop_click['uid'] = shop_click['uid'].astype(int)

In [ ]:

item_click_df = pd.DataFrame(columns=item_click.columns)
# shop_click_df = pd.DataFrame(columns=shop_click.columns)
for i in range(len(item_click)):
    item = item_click.iloc[i]
    updated = item['updated']        
    index = dates[dates['uid'] == item['uid']].index
    begin = dates.loc[index, 'begin'].values[0]
    end = dates.loc[index, 'ad_first_date'].values[0]
    if i % 1000 == 0:
        print(i/len(item_click))
    if updated >= begin and updated < end:
        item_click_df = item_click_df.append(item, ignore_index=True)
        

0.0
5.0837241064960895e-05
0.00010167448212992179
0.00015251172319488267
0.00020334896425984358
0.00025418620532480446
0.00030502344638976534
0.0003558606874547263
0.00040669792851968716
0.00045753516958464804
0.0005083724106496089
0.0005592096517145699
0.0006100468927795307
0.0006608841338444916
0.0007117213749094526
0.0007625586159744134
0.0008133958570393743
0.0008642330981043352
0.0009150703391692961
0.000965907580234257
0.0010167448212992179
0.0010675820623641788
0.0011184193034291397
0.0011692565444941007
0.0012200937855590614
0.0012709310266240223
0.0013217682676889833
0.0013726055087539442
0.0014234427498189051
0.0014742799908838658
0.0015251172319488268
0.0015759544730137877
0.0016267917140787486
0.0016776289551437096
0.0017284661962086703
0.0017793034372736312
0.0018301406783385922
0.0018809779194035531
0.001931815160468514
0.001982652401533475
0.0020334896425984357
0.002084326883663397
0.0021351641247283576
0.0021860013657933183
0.0022368386068582794
0.00228767584792324
0.00

0.01987736125639971
0.01992819849746467
0.019979035738529632
0.020029872979594594
0.020080710220659555
0.020131547461724512
0.020182384702789474
0.020233221943854435
0.020284059184919396
0.020334896425984357
0.020385733667049318
0.02043657090811428
0.02048740814917924
0.0205382453902442
0.020589082631309163
0.020639919872374124
0.020690757113439085
0.020741594354504046
0.020792431595569007
0.020843268836633965
0.020894106077698926
0.020944943318763887
0.02099578055982885
0.02104661780089381
0.02109745504195877
0.021148292283023732
0.021199129524088693
0.021249966765153654
0.021300804006218615
0.021351641247283577
0.021402478488348538
0.0214533157294135
0.021504152970478457
0.021554990211543418
0.02160582745260838
0.02165666469367334
0.0217075019347383
0.021758339175803262
0.021809176416868224
0.021860013657933185
0.021910850898998146
0.021961688140063107
0.022012525381128068
0.02206336262219303
0.02211419986325799
0.02216503710432295
0.02221587434538791
0.02226671158645287
0.0223175488

0.040263094923449025
0.04031393216451399
0.04036476940557895
0.04041560664664391
0.04046644388770887
0.040517281128773834
0.04056811836983879
0.040618955610903756
0.040669792851968714
0.04072063009303368
0.040771467334098636
0.0408223045751636
0.04087314181622856
0.04092397905729352
0.04097481629835848
0.04102565353942344
0.0410764907804884
0.04112732802155336
0.041178165262618326
0.04122900250368328
0.04127983974474825
0.041330676985813206
0.04138151422687817
0.04143235146794313
0.04148318870900809
0.04153402595007305
0.041584863191138015
0.04163570043220297
0.04168653767326793
0.041737374914332895
0.04178821215539785
0.04183904939646282
0.041889886637527775
0.04194072387859274
0.0419915611196577
0.04204239836072266
0.04209323560178762
0.042144072842852584
0.04219491008391754
0.042245747324982506
0.042296584566047464
0.04234742180711242
0.042398259048177386
0.042449096289242344
0.04249993353030731
0.042550770771372266
0.04260160801243723
0.04265244525350219
0.04270328249456715
0.04275

0.06085217755475819
0.06090301479582315
0.06095385203688811
0.06100468927795307
0.061055526519018036
0.06110636376008299
0.06115720100114796
0.061208038242212916
0.06125887548327788
0.06130971272434284
0.0613605499654078
0.06141138720647276
0.06146222444753772
0.06151306168860268
0.06156389892966764
0.061614736170732605
0.06166557341179756
0.06171641065286253
0.061767247893927485
0.06181808513499245
0.06186892237605741
0.06191975961712237
0.06197059685818733
0.062021434099252294
0.06207227134031725
0.06212310858138221
0.062173945822447174
0.06222478306351213
0.062275620304577096
0.062326457545642054
0.06237729478670702
0.062428132027771976
0.06247896926883694
0.0625298065099019
0.06258064375096686
0.06263148099203182
0.06268231823309678
0.06273315547416175
0.06278399271522671
0.06283482995629167
0.06288566719735662
0.06293650443842158
0.06298734167948655
0.06303817892055151
0.06308901616161647
0.06313985340268143
0.0631906906437464
0.06324152788481135
0.06329236512587631
0.063343202366

0.08184795811458705
0.081898795355652
0.08194963259671696
0.08200046983778192
0.08205130707884688
0.08210214431991185
0.0821529815609768
0.08220381880204176
0.08225465604310672
0.0823054932841717
0.08235633052523665
0.08240716776630161
0.08245800500736657
0.08250884224843154
0.0825596794894965
0.08261051673056145
0.08266135397162641
0.08271219121269137
0.08276302845375634
0.0828138656948213
0.08286470293588626
0.08291554017695121
0.08296637741801619
0.08301721465908114
0.0830680519001461
0.08311888914121106
0.08316972638227603
0.08322056362334099
0.08327140086440595
0.0833222381054709
0.08337307534653586
0.08342391258760083
0.08347474982866579
0.08352558706973075
0.0835764243107957
0.08362726155186068
0.08367809879292563
0.08372893603399059
0.08377977327505555
0.08383061051612052
0.08388144775718548
0.08393228499825044
0.0839831222393154
0.08403395948038035
0.08408479672144532
0.08413563396251028
0.08418647120357524
0.0842373084446402
0.08428814568570517
0.08433898292677013
0.084389820

...

0.1450894859993984
0.14514032324046336
0.1451911604815283
0.14524199772259327
0.14529283496365825
0.1453436722047232
0.14539450944578816
0.1454453466868531
0.14549618392791808
0.14554702116898305
0.145597858410048
0.14564869565111296
0.14569953289217794
0.14575037013324288
0.14580120737430785
0.1458520446153728
0.14590288185643777
0.14595371909750274
0.14600455633856768
0.14605539357963265
0.1461062308206976
0.14615706806176257
0.14620790530282754
0.14625874254389248
0.14630957978495746
0.14636041702602243
0.14641125426708737
0.14646209150815234
0.14651292874921729
0.14656376599028226
0.14661460323134723
0.14666544047241217
0.14671627771347714
0.1467671149545421
0.14681795219560706
0.14686878943667203
0.14691962667773698
0.14697046391880195
0.14702130115986692
0.14707213840093186
0.14712297564199683
0.14717381288306178
0.14722465012412675
0.14727548736519172
0.14732632460625666
0.14737716184732164
0.14742799908838658
0.14747883632945155
0.14752967357051652
0.14758051081158147
0.1476313

0.16623777828242212
0.1662886155234871
0.16633945276455206
0.166390290005617
0.16644112724668197
0.16649196448774692
0.1665428017288119
0.16659363896987686
0.1666444762109418
0.16669531345200678
0.16674615069307172
0.1667969879341367
0.16684782517520166
0.1668986624162666
0.16694949965733158
0.16700033689839655
0.1670511741394615
0.16710201138052647
0.1671528486215914
0.16720368586265638
0.16725452310372135
0.1673053603447863
0.16735619758585127
0.1674070348269162
0.16745787206798118
0.16750870930904616
0.1675595465501111
0.16761038379117607
0.16766122103224104
0.167712058273306
0.16776289551437096
0.1678137327554359
0.16786456999650087
0.16791540723756584
0.1679662444786308
0.16801708171969576
0.1680679189607607
0.16811875620182568
0.16816959344289065
0.1682204306839556
0.16827126792502056
0.16832210516608553
0.16837294240715048
0.16842377964821545
0.1684746168892804
0.16852545413034536
0.16857629137141034
0.16862712861247528
0.16867796585354025
0.1687288030946052
0.16877964033567017


0.18738607056544584
0.18743690780651082
0.1874877450475758
0.18753858228864073
0.1875894195297057
0.18764025677077065
0.18769109401183562
0.1877419312529006
0.18779276849396553
0.1878436057350305
0.18789444297609548
0.18794528021716042
0.1879961174582254
0.18804695469929034
0.1880977919403553
0.18814862918142028
0.18819946642248522
0.1882503036635502
0.18830114090461514
0.1883519781456801
0.18840281538674508
0.18845365262781003
0.188504489868875
0.18855532710993997
0.1886061643510049
0.18865700159206988
0.18870783883313483
0.1887586760741998
0.18880951331526477
0.18886035055632971
0.1889111877973947
0.18896202503845963
0.1890128622795246
0.18906369952058957
0.18911453676165452
0.1891653740027195
0.18921621124378446
0.1892670484848494
0.18931788572591438
0.18936872296697932
0.1894195602080443
0.18947039744910926
0.1895212346901742
0.18957207193123918
0.18962290917230412
0.1896737464133691
0.18972458365443406
0.189775420895499
0.18982625813656398
0.18987709537762895
0.1899279326186939
0.

0.2085343628484696
0.20858520008953454
0.20863603733059952
0.20868687457166446
0.20873771181272943
0.2087885490537944
0.20883938629485935
0.20889022353592432
0.20894106077698926
0.20899189801805423
0.2090427352591192
0.20909357250018415
0.20914440974124912
0.2091952469823141
0.20924608422337904
0.209296921464444
0.20934775870550895
0.20939859594657392
0.2094494331876389
0.20950027042870384
0.2095511076697688
0.20960194491083375
0.20965278215189873
0.2097036193929637
0.20975445663402864
0.2098052938750936
0.20985613111615858
0.20990696835722353
0.2099578055982885
0.21000864283935344
0.21005948008041841
0.2101103173214834
0.21016115456254833
0.2102119918036133
0.21026282904467825
0.21031366628574322
0.2103645035268082
0.21041534076787313
0.2104661780089381
0.21051701525000308
0.21056785249106802
0.210618689732133
0.21066952697319793
0.2107203642142629
0.21077120145532788
0.21082203869639282
0.2108728759374578
0.21092371317852274
0.2109745504195877
0.21102538766065268
0.21107622490171762


0.22973349237255827
0.22978432961362324
0.22983516685468822
0.22988600409575316
0.22993684133681813
0.22998767857788308
0.23003851581894805
0.23008935306001302
0.23014019030107796
0.23019102754214293
0.23024186478320788
0.23029270202427285
0.23034353926533782
0.23039437650640276
0.23044521374746774
0.2304960509885327
0.23054688822959765
0.23059772547066262
0.23064856271172757
0.23069939995279254
0.2307502371938575
0.23080107443492245
0.23085191167598743
0.23090274891705237
0.23095358615811734
0.2310044233991823
0.23105526064024726
0.23110609788131223
0.2311569351223772
0.23120777236344214
0.23125860960450711
0.23130944684557206
0.23136028408663703
0.231411121327702
0.23146195856876695
0.23151279580983192
0.23156363305089686
0.23161447029196183
0.2316653075330268
0.23171614477409175
0.23176698201515672
0.2318178192562217
0.23186865649728663
0.2319194937383516
0.23197033097941655
0.23202116822048152
0.2320720054615465
0.23212284270261144
0.2321736799436764
0.23222451718474135
0.232275354

In [ ]:
### for i in range(len(shop_click)):
    shop = shop_click.iloc[i]
    updated = shop['updated']
    index = dates[dates['uid'] == item['uid']].index
    begin = dates.loc[index, 'begin'].values[0]
    end = dates.loc[index, 'ad_first_date'].values[0]
    if i % 1000 == 0:
        print(i/len(shop_click))
    if updated >= begin and updated < end:
        shop_click_df = shop_click_df.append(shop, ignore_index=True)

In [167]:
shop_click_df

,id,uid,updated
0,2214487945,6886070,2018-12-28 15:41:34
1,2214491939,5409692,2018-12-28 15:43:34
2,2214493875,2683035,2018-12-28 15:44:53
3,2214494293,3017955,2018-12-28 15:44:53
4,2214494751,3017955,2018-12-28 15:45:46
...,...,...,...
591423,2397789115,3017955,2019-03-28 15:39:31
591424,2397790367,3017955,2019-03-28 15:39:31
591425,2397789015,6788274,2019-03-28 15:39:52
591426,2397790271,6788274,2019-03-28 15:39:52


In [169]:
shop_click_df.to_csv('shop_click_df.csv')